# Outline

- train word embeddings by using `matichon.json`
- use gensim `Word2Vec` model
- find similar words, calculate cosine similarity

# Import 

In [4]:
import pandas as pd
import numpy as np
import re, emoji, urllib, html
from gensim.models import Word2Vec
from pythainlp.tokenize import word_tokenize

- custom tokenization function
- remove all quatations and shrink newlines `\n` and white spaces 

In [5]:
def my_tokenize(text):
    ### REMOVE URL ###
    text = html.unescape(urllib.parse.unquote(text)) # unescape for unicode, unquote for escaped URL
    text = re.sub(r'https?.+?(?:\s|$)', '', text) # remove URL link
    ### REMOVE EMOJI ###
    text = emoji.replace_emoji(text) # remove emoji
    ### REPLACE ###
    text = re.sub(r'[“”„\"]', '', text) # remove double quotations
    text = re.sub(r'[‘’′′′′`\']', '', text) # remove single quotations
    text = re.sub(r'[\n\t\u00a0\xa0\u3000\u2002-\u200a\u202f]+', ' ', text) # shrink whitespaces e.g. good  boy -> good boy
    text = re.sub(r'[\r\u200b\ufeff]+', '', text) # remove non-breaking space
    text = re.sub(r'เเ', 'แ', text)
    ### SHRINK SOME REDUPLICATION ###
    text = re.sub(r'าา+', 'า', text)
    text = re.sub(r'ยย+', 'ย', text)
    text = re.sub(r'ๆๆ+', 'ๆ', text)
    text = re.sub(r'ะะ+', 'ะ', text)
    ### am ###
    text = re.sub(r'ํา','ำ', text) # o + า -> ำ
    text = re.sub(r'\u0E33([\u0E48\u0E49\u0E4A\u0E4B])', r'\1'+'\u0E33', text) # am + tone -> tone + am
    ### TOKENIZE AND FILTERING ###
    tokens = word_tokenize(text, keep_whitespace=False)
    tokens = [token.strip('(').strip(')') for token in tokens]
    tokens = [token for token in tokens if re.match(r'[0-9A-zก-ไ][0-9A-zก-๙\.]*', token)]
    return tokens

# Load data

- need only article, drop the other columns
- tokenize article and store as list of tokens

In [6]:
df = pd.read_json('data/matichon.json')
df

,headline,article,date,category,url,id
0,ซาอุฯจ่อเปิดไฟเขียวให้สิทธิหญิงม่าย-หย่าร้างปก...,(2 ธ.ค.58) หนังสือพิมพ์อัล ริยาดของทางการซาอุด...,2015-12-04 03:35:18,foreign,https://www.matichon.co.th/foreign/news_293,293
1,"""ไก่อู""ชี้ ตู่-เต้น ไม่ได้มีหน้าที่ตรวจสอบทุจร...","""บิ๊กป้อม"" แจง ครม. มีความพยายามยุยงปลุกปั่นให...",2015-12-04 04:10:49,politics,https://www.matichon.co.th/politics/news_329,329
2,"เปิดใจ ""โบว์ แวนดา"" ระหว่างรอยิ้มได้เต็มที่ในว...",แม้จะทำหน้าที่ภรรยาที่ดีมาเฝ้าปอ – ทฤษฎี สหวงษ...,2015-12-04 06:30:11,entertainment,https://www.matichon.co.th/entertainment/news_375,375
3,"""นาย ณภัทร"" ปลื้มคนชมแชมป์ขึ้นปกนิตยสารแห่งปี ...",กลายเป็นดาราหนุ่มเนื้อหอมแฟนคลับแน่น กระแสมาแร...,2015-12-04 07:10:26,entertainment,https://www.matichon.co.th/entertainment/news_393,393
4,"คอแทบหัก! แม่ยกแห่คล้องพวงมาลัยักษ์ ""บอย ศิริช...",แสดงดีจนเป็นที่ถูกอกถูกใจแฟนคลับ จนได้รับพวงมา...,2015-12-05 05:26:20,entertainment,https://www.matichon.co.th/entertainment/news_445,445
...,...,...,...,...,...,...
17104,โบว์ ณัฏฐา แจ้งความ พล.ต.อ.ศรีวราห์ ถูกพาดพิง...,เมื่อวันที่ 5 ก.ค. ที่ สน.พญาไท น.ส.ณัฏฐา มหัท...,2018-07-05 13:25:45,politics,https://www.matichon.co.th/politics/news_1029607,1029607
17105,ภาพบรรยากาศ ขุดทางระบายน้ำ เร่งนำ 13 ชีวิตออกจ...,วันที่ 5 กรกฎาคม เจ้าหน้าที่ขุดทางระบายน้ำที่ด...,2018-07-05 13:33:10,region,https://www.matichon.co.th/region/news_1029619,1029619
17106,สนช.ผ่านพ.ร.บ.สงฆ์ 3 วาระรวด พระมหากษัตริย์ทรง...,"สนช.ผ่าน พ.ร.บ.สงฆ์ 3 วาระรวด ""วิษณุ"" แจงสาระส...",2018-07-05 13:33:27,politics,https://www.matichon.co.th/politics/news_1029636,1029636
17107,นานาทรรศนะเพิ่มค่าปรับหมอ 5ล้านบ. สกัดเบี้ยว...,หมายเหตุ – จากกรณีที่ กระทรวงศึกษาธิการ (ศธ.) ...,2018-07-05 13:53:26,education,https://www.matichon.co.th/education/news_1029668,1029668


In [7]:
## drop column except for article
df.drop(columns=['date','headline','url','id'], inplace=True)

## tokenize
df['article_tokens'] = df['article'].apply(my_tokenize)

df

,article,category,article_tokens
0,(2 ธ.ค.58) หนังสือพิมพ์อัล ริยาดของทางการซาอุด...,foreign,"[2, ธ.ค., 58, หนังสือพิมพ์, อัล, ริยาด, ของ, ท..."
1,"""บิ๊กป้อม"" แจง ครม. มีความพยายามยุยงปลุกปั่นให...",politics,"[บิ๊ก, ป้อม, แจง, ครม., มี, ความพยายาม, ยุยง, ..."
2,แม้จะทำหน้าที่ภรรยาที่ดีมาเฝ้าปอ – ทฤษฎี สหวงษ...,entertainment,"[แม้, จะ, ทำหน้าที่, ภรรยา, ที่, ดี, มา, เฝ้า,..."
3,กลายเป็นดาราหนุ่มเนื้อหอมแฟนคลับแน่น กระแสมาแร...,entertainment,"[กลายเป็น, ดารา, หนุ่ม, เนื้อ, หอม, แฟนคลับ, แ..."
4,แสดงดีจนเป็นที่ถูกอกถูกใจแฟนคลับ จนได้รับพวงมา...,entertainment,"[แสดง, ดี, จน, เป็นที่, ถูกอกถูกใจ, แฟนคลับ, จ..."
...,...,...,...
17104,เมื่อวันที่ 5 ก.ค. ที่ สน.พญาไท น.ส.ณัฏฐา มหัท...,politics,"[เมื่อ, วันที่, 5, ก.ค., ที่, สน., พญาไท, น.ส...."
17105,วันที่ 5 กรกฎาคม เจ้าหน้าที่ขุดทางระบายน้ำที่ด...,region,"[วันที่, 5, กรกฎาคม, เจ้าหน้าที่, ขุด, ทาง, ระ..."
17106,"สนช.ผ่าน พ.ร.บ.สงฆ์ 3 วาระรวด ""วิษณุ"" แจงสาระส...",politics,"[สนช, ผ่าน, พ.ร.บ., สงฆ์, 3, วาระ, รวด, วิษณุ,..."
17107,หมายเหตุ – จากกรณีที่ กระทรวงศึกษาธิการ (ศธ.) ...,education,"[หมายเหตุ, จาก, กรณี, ที่, กระทรวงศึกษาธิการ, ..."


# Fit model with `gensim`

how to use :
[https://radimrehurek.com/gensim/models/word2vec.html](https://radimrehurek.com/gensim/models/word2vec.html)

- input of `Word2Vec` must be **list of list of tokens**
- `vector_size` : Dimensionality of the word vectors (usually 100-300)
- `window` : Maximum distance between the current and predicted word within a sentence
- `min_count` : Ignores all words with total frequency lower than this
- `sg` : Training algorithm: 1 for skip-gram; otherwise CBOW. (skip-gram is recommended)
- `epoch` : Number of iterations (epochs) over the corpus

In [8]:
## fit
## epoch=30, it may take over 5 minutes
model = Word2Vec(sentences=df['article_tokens'], vector_size=100, window=5, min_count=3, sg=1, epochs=30)

# Word Embeddings

- `model.wv.most_similar(word, topn=xx)` gives the top N words with high cosine similarity 


In [9]:
model.wv.most_similar('อร่อย', topn=10)

[('รสชาติ', 0.7745187878608704),
 ('กลมกล่อม', 0.7744733095169067),
 ('เมนู', 0.7495547533035278),
 ('ลิ้มลอง', 0.7371513247489929),
 ('โฮมเมด', 0.7112072110176086),
 ('ถูกปาก', 0.7110448479652405),
 ('กรุบ', 0.7027080059051514),
 ('จุใจ', 0.6983063817024231),
 ('ต้มยำ', 0.6966326832771301),
 ('สารพัน', 0.6920511722564697)]

- `model.wv.similarity(word1, word2)` calculates the cosine similarity of the 2 words

In [10]:
## antonym is also similar word
model.wv.similarity('ผู้ชาย', 'ผู้หญิง')

0.7978807

- `model.wv` behaves like a dictionary
- e.g. `model.wv[word]` gives the vector of the word

In [11]:
model.wv['อร่อย']

array([-1.1989086 , -0.14565909,  0.55839413, -0.8389895 , -0.05820602,
       -0.08373562,  0.22320378,  0.6289437 ,  0.5621025 , -0.39127314,
        0.07595494, -0.46179837, -0.19773336,  0.41880792,  0.26940152,
       -0.24190451, -0.22895698, -0.20063105, -0.23822665, -0.09482013,
        0.5070089 , -0.2610729 ,  0.6435401 , -0.35011357,  0.2705363 ,
       -0.4665677 , -0.01413696,  0.6545926 ,  0.27621782, -0.30576956,
        0.5271521 ,  0.7724907 , -0.09834579, -0.28051147, -0.1531951 ,
       -0.4112404 ,  0.3454889 , -0.5467533 ,  0.42288125, -0.11837536,
       -0.0491749 ,  0.10821851, -0.04626464, -1.0258904 ,  0.08398037,
        0.4200964 , -0.31281748,  0.3837073 ,  0.12394694,  0.28968418,
        0.3444184 ,  0.21646978, -1.0438312 , -0.33100352,  0.18724349,
        0.41513944, -0.24457207,  0.39815798,  0.09403691,  0.44299936,
       -0.13031356,  0.16737594, -0.37838024,  0.22873043,  0.04892971,
        0.16236168,  0.20507362,  0.5686902 , -0.08086511,  0.43

- `model.wv.most_similar()` can add/subtract vectors
- use argument `positive` and `negative`, e.g. `model.wv.most_similar(positive=[w1, w2], negative=[w3])`


~~~python
'ปักกิ่ง' - 'จีน' + 'ญี่ปุ่น' : model.wv.most_similar(positive=['ปักกิ่ง', 'ญี่ปุ่น'], negative=['จีน'])
~~~

In [12]:
## ปักกิ่ง - จีน + ญี่ปุ่น
model.wv.most_similar(positive=['ปักกิ่ง', 'ญี่ปุ่น'], negative=['จีน'])

[('โตเกียว', 0.6306976079940796),
 ('กรุง', 0.580376148223877),
 ('เบอร์ลิน', 0.528657853603363),
 ('อาบูดาบี', 0.5247726440429688),
 ('จาการ์ตา', 0.5111782550811768),
 ('สหรัฐอาหรับเอมิเรตส์', 0.5092828273773193),
 ('ฮอกไกโด', 0.5076935291290283),
 ('มอสโก', 0.5054842829704285),
 ('ฮาเนดะ', 0.5046287775039673),
 ('บาหลี', 0.5031425356864929)]

# save & load model

In [13]:
## save the model
## the same model is in `data` folder
model.save("data/word2vec_matichon.model")

In [15]:
## load pre-trained model 
model_loaded = Word2Vec.load("data/word2vec_matichon.model")
model_loaded.wv.most_similar('สวย')

[('เซ็กซี่', 0.6959749460220337),
 ('น่ารัก', 0.671596348285675),
 ('ซิกซ์แพ็ก', 0.6411164999008179),
 ('สวยงาม', 0.6394573450088501),
 ('เปล่งประกาย', 0.6126203536987305),
 ('ลุค', 0.6113107204437256),
 ('สะบึม', 0.6079009771347046),
 ('สะพรั่ง', 0.5995209217071533),
 ('บิกินี่', 0.5970008373260498),
 ('หน้าหวาน', 0.594933032989502)]

# Visualization

- go to https://projector.tensorflow.org/
- prepare 2 files:
    1. `tsv` file of all vectors (without index, without header)
    2. `tsv` file of labels (without index, with header)
- click "Load" and upload files
- it shows neighbor words in 3D plot

In [21]:
## make dataframe 
wv_df = pd.DataFrame(model.wv.vectors, columns=np.arange(1, 101)) # value of vectors
wv_df['word'] = model.wv.index_to_key # all vocabs
wv_df['count'] = wv_df['word'].apply(lambda x: model.wv.get_vecattr(x, 'count')) # get word count of each word
wv_df['freq/10M'] = (wv_df['count'] * 10000000 / wv_df['count'].sum()).round(1) # calculate word frequency in 10M words

wv_df.head()

,1,2,3,4,5,6,7,8,9,10,...,94,95,96,97,98,99,100,word,count,freq/10M
0,0.147107,0.205480,-0.080089,0.009322,-0.110583,-0.359767,0.186321,0.264107,-0.145018,0.053605,...,0.146914,0.328782,0.044164,-0.107805,-0.103107,0.094986,-0.065493,ที่,124850,202167.5
1,-0.076751,0.106975,-0.088567,-0.155876,0.131924,-0.413932,0.050694,0.405921,-0.200009,0.023447,...,-0.287021,0.205203,0.251952,0.006049,-0.087677,-0.002882,0.151624,และ,118744,192280.1
2,0.248766,0.236299,-0.139819,-0.158100,0.108476,-0.211760,0.100985,0.349802,-0.093654,0.041861,...,-0.271012,0.309146,0.211601,-0.102019,-0.161443,-0.143182,-0.138014,ใน,98174,158971.5
3,0.017607,0.330226,-0.128629,0.018312,0.143110,-0.360320,-0.297999,0.193926,-0.059721,-0.189001,...,-0.288652,0.388178,-0.151375,0.011995,0.012237,-0.119617,-0.043767,มี,84284,136479.6
4,0.009582,0.126914,0.096425,-0.209569,0.019580,-0.174211,-0.008858,0.744829,-0.203719,-0.405993,...,-0.447429,0.289066,0.165223,0.089111,0.048112,-0.004735,-0.147302,การ,74873,121240.6


In [22]:
## save to tsv file
## vector tsv files (without index, header)
wv_df.drop(columns=['word','count','freq/10M'], axis=1).to_csv('data/wv.tsv', sep='\t', index=False, header=False)

## label tsv files (with label `word` and `word frequency`)
wv_df[['word','freq/10M']].to_csv('data/wv_label.tsv', sep='\t', index=False)

![wv_visualization](image/wv_visualization.png)